In [1]:
import pandas as pd
import numpy as np
import datetime as datetime

In [2]:
keys = ['8to9', '9to10', '10to11', '11to12','12to13','13to14','14to15','15to16','16to17','17to18','18to19','19to20']

In [3]:
# read data
random_ema = pd.read_csv('random-ema.csv')
eod_ema = pd.read_csv('eod-ema.csv')

random_ema_alternative = pd.read_csv('random-ema-alternative.csv')
eod_ema_alternative = pd.read_csv('eod-ema-alternative.csv')

random_ema_backup = pd.read_csv('random-ema-backup.csv')
eod_ema_backup = pd.read_csv('eod-ema-backup.csv')

In [7]:
def random_vs_eod(random_ema, eod_ema):
    days_smoked = {}
    for index, row in eod_ema.iterrows():
        if row['status'] == "MISSED":
            continue
        for i in keys:
            if row[i] == 1:
                try:
                    time = datetime.datetime.strptime(row['date'], '%m/%d/%Y %H:%M')
                except:
                    time = datetime.datetime.strptime(row['date'], '%Y-%m-%d %H:%M:%S')
                date = (time.year, time.month, time.day)
                if time.hour  == 0 or time.hour == 1:
                    date = (time.year, time.month, time.day-1)
                if row['participant_id'] not in days_smoked:
                    days_smoked[row['participant_id']] = set()
                days_smoked[row['participant_id']].add(date)
                break

    count = 0 
    missing = 0
    missing_days = 0
    lst = []
    events = set()
    for index, row in random_ema.iterrows():
        if row['status'] == "MISSED":
            continue
        if row['smoke'] != "Yes":
            continue
        count += 1
        try:
            time = datetime.datetime.strptime(row['date'], '%m/%d/%y %H:%M')
        except:
            time = datetime.datetime.strptime(row['date'], '%Y-%m-%d %H:%M:%S')
        date = (time.year, time.month, time.day)
        if row['participant_id'] not in days_smoked:
            continue
        if date not in days_smoked[row['participant_id']]:
            missing += 1
            if date not in events:
                missing_days += 1
                events.add(date)
            lst.append(index)

    missing_in_eod = random_ema.ix[lst]
    print("Random vs. EOD EMA inconsistency percentage by entries is: ", missing/float(count))
    print("Random vs. EOD EMA inconsistency percentage by day is: ", missing_days/float(count))
    return missing_in_eod

In [8]:
random_vs_eod(random_ema, eod_ema)

Random vs. EOD EMA inconsistency percentage by entries is:  0.205607476635514
Random vs. EOD EMA inconsistency percentage by day is:  0.1588785046728972


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,status,smoke,when_smoke,eat,when_eat,drink,when_drink,urge,cheerful,happy,...,see_or_smell,access,smoking_location,participant_id,timestamp,offset,date,hour,minute,day_of_week
199,COMPLETED,Yes,20 - 39 Minutes,No,None,Yes,1 - 19 Minutes,4,4,4,...,3,4,3,211,1508164102983,-360,2017-10-16 10:28:22,10,28,Monday
260,COMPLETED,Yes,40 - 59 Minutes,No,None,No,None,3,3,3,...,3,3,3,214,1509376216016,-360,2017-10-30 11:10:16,11,10,Monday
350,COMPLETED,Yes,40 - 59 Minutes,No,None,Yes,20 - 39 Minutes,1,6,6,...,3,4,3,217,1509459513530,-360,2017-10-31 10:18:33,10,18,Tuesday
352,COMPLETED,Yes,20 - 39 Minutes,No,None,Yes,1 - 19 Minutes,3,4,4,...,4,3,3,217,1509494410766,-360,2017-10-31 20:00:10,20,0,Tuesday
382,COMPLETED,Yes,20 - 39 Minutes,No,None,Yes,20 - 39 Minutes,5,4,4,...,3,4,4,218,1509803438749,-360,2017-11-04 09:50:38,9,50,Saturday
384,COMPLETED,Yes,1 - 19 Minutes,Yes,20 - 39 Minutes,Yes,20 - 39 Minutes,4,4,4,...,4,4,4,218,1509827560236,-360,2017-11-04 16:32:40,16,32,Saturday
391,COMPLETED,Yes,20 - 39 Minutes,Yes,20 - 39 Minutes,Yes,20 - 39 Minutes,2,5,5,...,3,3,4,218,1510068314373,-360,2017-11-07 10:25:14,10,25,Tuesday
466,COMPLETED,Yes,1 - 19 Minutes,Yes,1 - 19 Minutes,Yes,1 - 19 Minutes,1,3,3,...,3,4,4,220,1511879209322,-360,2017-11-28 09:26:49,9,26,Tuesday
467,COMPLETED,Yes,20 - 39 Minutes,Yes,20 - 39 Minutes,Yes,20 - 39 Minutes,5,3,3,...,3,4,4,220,1511894551675,-360,2017-11-28 13:42:31,13,42,Tuesday
472,COMPLETED,Yes,1 - 19 Minutes,Yes,1 - 19 Minutes,Yes,1 - 19 Minutes,1,4,4,...,4,4,4,220,1512049529891,-360,2017-11-30 08:45:29,8,45,Thursday


In [9]:
random_vs_eod(random_ema_alternative, eod_ema_alternative)

Random vs. EOD EMA inconsistency percentage by entries is:  0.23684210526315788
Random vs. EOD EMA inconsistency percentage by day is:  0.15789473684210525


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,status,smoke,when_smoke,eat,when_eat,drink,when_drink,urge,cheerful,happy,...,sad,see_or_smell,access,smoking_location,participant_id,timestamp,date,hour,minute,day_of_week
46,COMPLETED,Yes,1 - 19 Minutes,Yes,1 - 19 Minutes,Yes,1 - 19 Minutes,4.0,4.0,4.0,...,2.0,3.0,3.0,3.0,224,1516638297084,2018-01-22 11:24:57,11,24,Monday
49,COMPLETED,Yes,20 - 39 Minutes,Yes,20 - 39 Minutes,Yes,40 - 59 Minutes,1.0,3.0,3.0,...,2.0,4.0,4.0,4.0,224,1516721723160,2018-01-23 10:35:23,10,35,Tuesday
50,COMPLETED,Yes,20 - 39 Minutes,No,None,Yes,20 - 39 Minutes,1.0,1.0,1.0,...,2.0,4.0,4.0,4.0,224,1516736644832,2018-01-23 14:44:04,14,44,Tuesday
51,COMPLETED,Yes,40 - 59 Minutes,Yes,40 - 59 Minutes,Yes,20 - 39 Minutes,2.0,1.0,1.0,...,3.0,3.0,3.0,3.0,224,1516752084620,2018-01-23 19:01:24,19,1,Tuesday
71,COMPLETED,Yes,40 - 59 Minutes,Yes,20 - 39 Minutes,Yes,60 - 79 Minutes,2.0,1.0,1.0,...,1.0,4.0,4.0,4.0,224,1517509909817,2018-02-01 13:31:49,13,31,Thursday
72,COMPLETED,Yes,20 - 39 Minutes,No,None,No,None,1.0,1.0,1.0,...,1.0,4.0,4.0,4.0,224,1517525353469,2018-02-01 17:49:13,17,49,Thursday
88,COMPLETED,Yes,60 - 79 Minutes,Yes,40 - 59 Minutes,Yes,60 - 79 Minutes,4.0,3.0,3.0,...,3.0,3.0,3.0,4.0,225,1517411460652,2018-01-31 10:11:00,10,11,Wednesday
90,COMPLETED,Yes,60 - 79 Minutes,No,None,Yes,1 - 19 Minutes,3.0,3.0,3.0,...,3.0,3.0,3.0,4.0,225,1517441260484,2018-01-31 18:27:40,18,27,Wednesday
93,COMPLETED,Yes,1 - 19 Minutes,No,None,Yes,1 - 19 Minutes,5.0,3.0,3.0,...,4.0,3.0,4.0,4.0,225,1517605954511,2018-02-02 16:12:34,16,12,Friday
94,COMPLETED,Yes,1 - 19 Minutes,Yes,1 - 19 Minutes,Yes,1 - 19 Minutes,4.0,3.0,3.0,...,3.0,3.0,4.0,4.0,225,1517614976885,2018-02-02 18:42:56,18,42,Friday


In [10]:
random_vs_eod(random_ema_backup, eod_ema_backup)

Random vs. EOD EMA inconsistency percentage by entries is:  0.17142857142857143
Random vs. EOD EMA inconsistency percentage by day is:  0.13333333333333333


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,status,smoke,when_smoke,eat,when_eat,drink,when_drink,urge,cheerful,happy,...,sad,see_or_smell,access,smoking_location,participant_id,timestamp,date,hour,minute,day_of_week
200,COMPLETED,Yes,20 - 39 Minutes,No,None,Yes,1 - 19 Minutes,4,4,4,...,4,3,4,3,211,1508164102983,2017-10-16 10:28:22,10,28,Monday
263,COMPLETED,Yes,40 - 59 Minutes,No,None,No,None,3,3,3,...,3,3,3,3,214,1509376216016,2017-10-30 11:10:16,11,10,Monday
356,COMPLETED,Yes,40 - 59 Minutes,No,None,Yes,20 - 39 Minutes,1,6,6,...,1,3,4,3,217,1509459513530,2017-10-31 10:18:33,10,18,Tuesday
358,COMPLETED,Yes,20 - 39 Minutes,No,None,Yes,1 - 19 Minutes,3,4,4,...,1,4,3,3,217,1509494410766,2017-10-31 20:00:10,20,0,Tuesday
388,COMPLETED,Yes,20 - 39 Minutes,No,None,Yes,20 - 39 Minutes,5,4,4,...,3,3,4,4,218,1509803438749,2017-11-04 09:50:38,9,50,Saturday
390,COMPLETED,Yes,1 - 19 Minutes,Yes,20 - 39 Minutes,Yes,20 - 39 Minutes,4,4,4,...,3,4,4,4,218,1509827560236,2017-11-04 16:32:40,16,32,Saturday
397,COMPLETED,Yes,20 - 39 Minutes,Yes,20 - 39 Minutes,Yes,20 - 39 Minutes,2,5,5,...,1,3,3,4,218,1510068314373,2017-11-07 10:25:14,10,25,Tuesday
474,COMPLETED,Yes,1 - 19 Minutes,Yes,1 - 19 Minutes,Yes,1 - 19 Minutes,1,3,3,...,4,3,4,4,220,1511879209322,2017-11-28 09:26:49,9,26,Tuesday
475,COMPLETED,Yes,20 - 39 Minutes,Yes,20 - 39 Minutes,Yes,20 - 39 Minutes,5,3,3,...,4,3,4,4,220,1511894551675,2017-11-28 13:42:31,13,42,Tuesday
480,COMPLETED,Yes,1 - 19 Minutes,Yes,1 - 19 Minutes,Yes,1 - 19 Minutes,1,4,4,...,3,4,4,4,220,1512049529891,2017-11-30 08:45:29,8,45,Thursday
